来源：https://wiseodd.github.io/techblog/2017/09/07/lda-gibbs/

In [ ]:
import numpy as np

### 介绍

LDA的目标：输入一些文档（以词袋模型，词之间的次序不重要），要推断这些文档的潜在主题。

设$i=\{1,\cdots,N_{D}\}$为文档的下标，$v=\{1,\cdots,N_{W}\}$为词的下标，$k=\{1,\cdots,N_{K}\}$为topic的下标。LDA假设：

$\pi_{i}\sim Dir(\pi_{i}|\alpha)$：即第i篇文档的topic，其分布来源于一个参数为$\alpha$的Dirichlet分布。

$z_{iv}\sim Mult(z_{iv}|\pi_{i})$：即第i篇文档的第v个词的topic，其分布来源于一个参数为$\pi_{i}$的多项式分布。

$b_{k}\sim Dir(b_{k}|\gamma)$：即第k个topic对应的词的分布，其分布来源于一个参数为$\gamma$的Dirichlet分布。

$y_{iv}\sim Mult(y_{iv}|z_{iv}=k,B)$：即第i篇文档的第v个词，其分布来源于其topic（$z_{iv}$）对应的词分布（$b_k$）。

### Inference

LDA的本质就是已知$y_{iv}$，反推出$z_{iv}$，$\pi_{i}$和$b_k$。

可以使用Gibbs Sampling算法推断这些变量。利用Gibbs Sampling的本质就是固定其他2个变量，然后利用条件概率估计第三个变量，这个过程重复进行直到收敛。三个条件概率分布如下：

![lda](LDA_cond_prob.PNG)

有了这些条件概率，便可以把它们放入Gibbs Sampling的框架中。

### Implementation

$z_{iv}$，$\pi_{i}$和$b_k$对应着三个矩阵：$Z_{N_{D}\times N_{W}}$，它为每篇文档中的每个词赋予了一个topic；$\Pi_{N_{D}\times N_{K}}$，它表示每篇文章对应的topic分布；$B_{N_{K}\times N_{W}}$，它表示每个topic对应的词的分布。

下面先初始化这三个矩阵：

In [ ]:
"""
    N_D, N_W, N_K在后面定义
"""

# Dirichlet priors
# 用来描述Dirichlet分布的宽窄
alpha = 1
gamma = 1

# Z := word topic assignment
Z = np.zeros(shape=[N_D, N_W])

for i in range(N_D):
    for l in range(N_W):
        Z[i, l] = np.random.randint(N_K)  # randomly assign word's topic

# Pi := document topic distribution
Pi = np.zeros([N_D, N_K])

for i in range(N_D):
    Pi[i] = np.random.dirichlet(alpha*np.ones(N_K))

# B := word topic distribution
B = np.zeros([N_K, N_W])

for k in range(N_K):
    B[k] = np.random.dirichlet(gamma*np.ones(N_W))

下面是Gibbs Sampling过程：

In [ ]:
# X变量为语料库，每行代表一篇文章，其中的每个元素代表一个词
# W变量为词库，其中每个元素代表一个词

for idx in range(1000):
    
    # 固定Pi和B，变化Z [N_D x N_W]
    # Sample from full conditional of Z
    for i in range(N_D):
        for v in range(N_W):
            """
                更新Z[i,v]，即词（i，v）所属的topic
                语料库中第（i，v）个词的topic服从多项式分布
                
                先计算该多项式分布的概率向量
                Pi[i]：第i篇文章的topic分布向量
                B[:,X[i,v]]：词（i，v）在各个topic上的得分
                p_iv：词（i，v）在各个topic上的可能性
                
                再从该分布中采样，更新Z[i,v]
            """
            p_iv = np.exp(np.log(Pi[i]) + np.log(B[:, X[i, v]]))
            p_iv /= np.sum(p_iv)

            Z[i, v] = np.random.multinomial(1, p_iv).argmax()
            
        
    # 固定Z和B，变化Pi [N_D x N_K]
    # Sample from full conditional of Pi
    for i in range(N_D):
        """
            更新Pi[i, :]，即文章i中各个topic的分布，其服从Dirichlet分布
            这里先计算真实计数m，加上伪计数alpha，得到后验分布的参数m+alpha
            
            m[k]的值即为文章i中topic属于k的词数
        """
        m = np.zeros(N_K)
        
        for k in range(N_K):
            m[k] = np.sum(Z[i] == k)
            
        Pi[i, :] = np.random.dirichlet(alpha + m)
    